In [0]:
# COMMAND ----------
import logging
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import DateType

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# COMMAND ----------
try:
    # Step 1: Data Loading
    logger.info("Loading data from Unity Catalog tables.")
    claims_df = spark.table("genai_demo.jnj.claims")
    demographics_df = spark.table("genai_demo.jnj.demographics")
    policy_df = spark.table("genai_demo.jnj.policy")
    scores_df = spark.table("genai_demo.jnj.scores")
    aiml_insights_df = spark.table("genai_demo.jnj.aiml_insights")

# COMMAND ----------
    # Step 2: Data Selection and Filtering
    logger.info("Selecting relevant fields from each DataFrame.")
    demographics_df = demographics_df.select("Customer_ID", "Customer_Name", "Date_of_Birth", "Gender", "Income_Level")
    claims_df = claims_df.select("Claim_ID", "Policy_ID", "Claim_Date", "Claim_Amount")
    policy_df = policy_df.select("Policy_ID", "Customer_ID", "Policy_Type", "Policy_Premium", "Total_Premium_Paid")

    # Convert date fields to DateType
    demographics_df = demographics_df.withColumn("Date_of_Birth", F.col("Date_of_Birth").cast(DateType()))
    claims_df = claims_df.withColumn("Claim_Date", F.col("Claim_Date").cast(DateType()))

# COMMAND ----------
    # Step 3: Data Integration and Joining
    logger.info("Joining DataFrames on common identifiers.")
    customer_policy_df = demographics_df.join(policy_df, "Customer_ID")
    customer_claims_df = customer_policy_df.join(claims_df, "Policy_ID")

# COMMAND ----------
    # Step 4: Data Aggregation
    logger.info("Computing aggregate metrics.")
    aggregated_df = customer_claims_df.groupBy("Customer_ID").agg(
        F.count("Claim_ID").alias("Total_Claims"),
        F.avg("Claim_Amount").alias("Average_Claim_Amount"),
        F.max("Claim_Date").alias("Recent_Claim_Date")
    )

# COMMAND ----------
    # Step 5: Custom Calculations
    logger.info("Implementing custom calculations for additional metrics.")
    final_df = aggregated_df.join(customer_policy_df, "Customer_ID").withColumn(
        "Age", F.datediff(F.current_date(), F.col("Date_of_Birth")) / 365
    ).withColumn(
        "Claim_To_Premium_Ratio", F.when(F.col("Total_Premium_Paid") != 0, F.col("Average_Claim_Amount") / F.col("Total_Premium_Paid")).otherwise(0)
    ).withColumn(
        "Claims_Per_Policy", F.when(F.col("Policy_Type").isNotNull(), F.col("Total_Claims") / F.count("Policy_ID").over(Window.partitionBy("Customer_ID"))).otherwise(0)
    ).withColumn(
        "Retention_Rate", F.lit(0.85)
    ).withColumn(
        "Cross_Sell_Opportunities", F.lit("Multi-Policy Discount, Home Coverage Add-on")
    ).withColumn(
        "Upsell_Potential", F.lit("Premium Vehicle Coverage")
    )

# COMMAND ----------
    # Step 6: Comprehensive Data Joining
    logger.info("Integrating additional insights from AI/ML data and scores.")
    final_df = final_df.join(scores_df, "Customer_ID").join(aiml_insights_df, "Customer_ID")

# COMMAND ----------
    # Step 7: Output to Customer 360 View
    logger.info("Writing the final DataFrame to the target table in Databricks.")
    # final_df.write.format("delta").mode("overwrite").saveAsTable("genai_demo.jnj.customer_360_view")
    final_df.show()
    logger.info("ETL process completed successfully.")

except Exception as e:
    logger.error("An error occurred during the ETL process.", exc_info=True)
